In [5]:
!python3 -m pip install pandas

     |████████████████████████████████| 11.3 MB 3.5 MB/s eta 0:00:01    |█████████████████████▏          | 7.5 MB 3.5 MB/s eta 0:00:02
     |████████████████████████████████| 17.0 MB 27.2 MB/s eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [61]:
# import libraries
import pandas as pd
import numpy as np
import re
print("test")

test


In [62]:
# load data 
git_repo = "https://raw.githubusercontent.com/rivergrove/Join-the-dark-side-of-chess/master/python_2021/"
file = "lichess_rivergrove_2021-08-09.csv"
user_name = "rivergrove"
df = pd.read_csv(git_repo + file, delimiter = "\t", header=None, names = ['col'])
df.head()

,col
0,"[Event ""Rated Rapid game""]"
1,"[Site ""https://lichess.org/MJtPxMic""]"
2,"[Date ""2021.06.12""]"
3,"[White ""rivergrove""]"
4,"[Black ""Ramses_1""]"


In [6]:
df.info

<bound method DataFrame.info of                                                      col
0                             [Event "Rated Rapid game"]
1                  [Site "https://lichess.org/MJtPxMic"]
2                                    [Date "2021.06.12"]
3                                   [White "rivergrove"]
4                                     [Black "Ramses_1"]
...                                                  ...
17217                              [TimeControl "600+0"]
17218                                        [ECO "D02"]
17219  [Opening "Queen's Pawn Game: Chigorin Variation"]
17220                             [Termination "Normal"]
17221  1. Nf3 { [%eval 0.16] } 1... d5 { [%eval 0.25]...

[17222 rows x 1 columns]>

In [63]:
# start row is every row with [Event...]
start_row = df[df['col'].str.contains('[Event ', regex=False)].index

In [64]:
if 'games' in globals():
    del games
    
for i in range(1,len(start_row)):
    
    if 'games' in globals():
        
        if "Horde" in df.iloc[start_row[i-1],0]:
            # skip Horde Games
            pass
        elif "Casual" in df.iloc[start_row[i-1],0]:
            temp = df.iloc[start_row[i-1]:start_row[i-1]+10,0]
            temp.loc[len(temp.index)] = [""] 
            temp.loc[len(temp.index)] = [""]
            temp = pd.DataFrame(temp).append(df.iloc[start_row[i-1]+10:start_row[i]]).T
            temp.columns = games.columns
            games = games.append(temp)
            pass
        elif len(games.columns) != (start_row[i]-start_row[i-1]):
            # skip error games
            print("row " + str(i-1) + ": column count does not align. There are " + str(len(games.columns)) + " in games and " + str(start_row[i]-start_row[i-1]) + " in temp.")
        else:
            temp = pd.DataFrame(df.iloc[start_row[i-1]:start_row[i],0]).T
            temp.columns = games.columns
            games = games.append(temp)
        
    else:
        
        if "Casual" in df.iloc[start_row[i-1],0]:
            temp = df.iloc[0:10,0]
            temp = temp.append([""])
            temp = temp.append([""])
            temp = temp.append(df.iloc[10:start_row[i]])
            games = pd.DataFrame(temp).T
        else: 
            games = pd.DataFrame(df.iloc[0:start_row[i],0]).T

row 764: column count does not align. There are 18 in games and 20 in temp.
row 937: column count does not align. There are 18 in games and 16 in temp.


In [65]:
# rename columns
games = games.rename(columns = {0: 'game_type', 1: 'url', 2:'date_utc', 3:'temp_white', 4:'temp_black', 5:'result', 7:'time_utc',8:'white_rating',9:'black_rating',10:'white_rating_diff',11:'black_rating_diff',12:'variant',13:'time_control',14:'opening_code',15:'opening_name',16:'termination_type',17:'moves'}, inplace = False)


In [66]:
# Pull value for game type
games['game_type'] = games['game_type'].str.split().str[-2]

In [67]:
# remove [], "", heading from values
def clean(text):
    return text.str.split(' ', 1).str[1].str.replace(']','').str.replace('"','')

games.iloc[:,1:17] = games.iloc[:,1:17].apply(clean)

/var/folders/_1/fhhdt3rx4b547dqcswnn523w0000gn/T/ipykernel_90702/3494051032.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  return text.str.split(' ', 1).str[1].str.replace(']','').str.replace('"','')


In [68]:
# remove column 6
games.drop(columns=[6])

,game_type,url,date_utc,temp_white,temp_black,result,time_utc,white_rating,black_rating,white_rating_diff,black_rating_diff,variant,time_control,opening_code,opening_name,termination_type,moves
col,Rapid,https://lichess.org/MJtPxMic,2021.06.12,rivergrove,Ramses_1,1-0,07:51:47,2031,2108,+8,-7,Standard,300+8,D15,Slav Defense: Schlechter Variation,Normal,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... d5 {...
col,Correspondence,https://lichess.org/X5pBsOro,2021.06.12,rivergrove,lichess AI level 6,1-0,07:01:26,1500,?,NaN,NaN,Standard,-,D07,"Queen's Gambit Declined: Chigorin Defense, Exc...",Normal,1. d4 { [%eval 0.0] } 1... d5 { [%eval 0.27] }...
col,Rapid,https://lichess.org/pYEblF9q,2021.06.09,pishraft00,rivergrove,0-1,07:44:09,1907,2027,-4,+4,Standard,300+5,D08,"Queen's Gambit Declined: Albin Countergambit, ...",Normal,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... d5 {...
col,Rapid,https://lichess.org/4YNJRTP4,2021.06.09,rivergrove,Hovhannes1982,1-0,07:09:47,2019,2076,+8,-6,Standard,300+8,C02,"French Defense: Advance Variation, Paulsen Attack",Normal,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... e6 {...
col,Rapid,https://lichess.org/5d84RUir,2021.06.08,kiba2015,rivergrove,0-1,07:18:53,1711,2017,-2,+2,Standard,300+8,B20,Sicilian Defense: Bowdler Attack,Normal,1. e4 { [%eval 0.24] [%clk 0:05:00] } 1... c5 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
col,Blitz,https://lichess.org/LnENU2NN,2016.04.18,tarunbahel,rivergrove,0-1,05:18:30,1619,1681,-8,+99,Standard,180+5,C70,"Ruy Lopez: Morphy Defense, Caro Variation",Normal,1. e4 { [%eval 0.22] } 1... e5 { [%eval 0.15] ...
col,Blitz,https://lichess.org/kyNvSaQD,2016.04.18,WHITETIGER13,rivergrove,1-0,05:09:40,1814,1681,NaN,NaN,Standard,300+0,B20,Sicilian Defense: Snyder Variation,Normal,1. e4 { [%eval 0.28] } 1... c5 { [%eval 0.33] ...
col,Blitz,https://lichess.org/5ZGew0zO,2016.04.18,rivergrove,rty666,1-0,05:03:09,1500,1507,+181,-7,Standard,300+0,A34,"English Opening: Symmetrical Variation, Normal...",Normal,1. c4 c5 2. Nc3 e5 3. Nf3 d6 4. d3 Bg4 5. e4 N...
col,Rapid,https://lichess.org/IGk0D1Jp,2016.04.18,rivergrove,wandi_chess,1-0,02:28:43,1869,1870,+101,-9,Standard,480+3,E20,Nimzo-Indian Defense,Normal,1. c4 Nf6 2. d4 e6 3. Nc3 Bb4 4. Bd2 h6 5. a3 ...


In [69]:
# reset index
games = games.reset_index(drop=True)
games.index

RangeIndex(start=0, stop=969, step=1)

In [70]:
user_name = 'rivergrove'
games['white'] = games['temp_white'] == user_name
# if 'white' == True, then opp_name = username else opp_name = temp_black
games['opp_name'] = np.where(games['white']==True, games['temp_black'], games['temp_white'])
# remove temp_black, temp_white, result
# result
games['outcome'] = np.select([(games['white']==True) & (games['result']=="1-0"), (games['white']==False) & (games['result']=="0-1"), games['result']=="1/2-1/2", (games['white']==False) & (games['result']=="1-0"), (games['white']==True) & (games['result']=="0-1")], ["win", "win", "draw", "loss", "loss"], default="error")
# opponent rating vs my rating
games['my_rating'] = np.where(games['white']==True, games['white_rating'], games['black_rating'])
games['opp_rating'] = np.where(games['white']==True, games['black_rating'], games['white_rating'])
games['my_rating_diff'] = np.where(games['white']==True, games['white_rating_diff'].str.replace('+','',regex=False), games['black_rating_diff'].str.replace('+','',regex=False))
games['opp_rating_diff'] = np.where(games['white']==True, games['black_rating_diff'].str.replace('+','',regex=False), games['white_rating_diff'].str.replace('+','',regex=False))


In [71]:
# remove extra date column
games = games.drop(6, 1)

/var/folders/_1/fhhdt3rx4b547dqcswnn523w0000gn/T/ipykernel_90702/787336042.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  games = games.drop(6, 1)


In [72]:
# remove extra columns
games.drop(['temp_white', 'temp_black','result','white_rating','black_rating','white_rating_diff','black_rating_diff'], axis=1, inplace=True)

In [73]:
# add seconds and increment

# remove [], "", heading from values
def clean_seconds(text):
    return str(text).split('+', 1)[0]
def clean_increment(text):
    return str(text).split('+', 1)[-1]

games['seconds'] = games.iloc[:,5].apply(clean_seconds)
games['increment'] = games.iloc[:,5].apply(clean_increment)

In [74]:
# add game_id as first column
games["id"] = games.index + 1

In [75]:
# reorder columns
games = games[["id","game_type","variant","date_utc","time_utc","seconds","increment","opening_code","opening_name","termination_type","url","white","opp_name","outcome","my_rating","opp_rating","my_rating_diff","opp_rating_diff","moves"]]

In [76]:
games.head(11)

,id,game_type,variant,date_utc,time_utc,seconds,increment,opening_code,opening_name,termination_type,url,white,opp_name,outcome,my_rating,opp_rating,my_rating_diff,opp_rating_diff,moves
0,1,Rapid,Standard,2021.06.12,07:51:47,300,8,D15,Slav Defense: Schlechter Variation,Normal,https://lichess.org/MJtPxMic,True,Ramses_1,win,2031,2108,8,-7,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... d5 {...
1,2,Correspondence,Standard,2021.06.12,07:01:26,-,-,D07,"Queen's Gambit Declined: Chigorin Defense, Exc...",Normal,https://lichess.org/X5pBsOro,True,lichess AI level 6,win,1500,?,NaN,NaN,1. d4 { [%eval 0.0] } 1... d5 { [%eval 0.27] }...
2,3,Rapid,Standard,2021.06.09,07:44:09,300,5,D08,"Queen's Gambit Declined: Albin Countergambit, ...",Normal,https://lichess.org/pYEblF9q,False,pishraft00,win,2027,1907,4,-4,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... d5 {...
3,4,Rapid,Standard,2021.06.09,07:09:47,300,8,C02,"French Defense: Advance Variation, Paulsen Attack",Normal,https://lichess.org/4YNJRTP4,True,Hovhannes1982,win,2019,2076,8,-6,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... e6 {...
4,5,Rapid,Standard,2021.06.08,07:18:53,300,8,B20,Sicilian Defense: Bowdler Attack,Normal,https://lichess.org/5d84RUir,False,kiba2015,win,2017,1711,2,-2,1. e4 { [%eval 0.24] [%clk 0:05:00] } 1... c5 ...
5,6,Rapid,Standard,2021.06.06,07:04:17,300,8,A40,Modern Defense,Normal,https://lichess.org/GdvABrX5,True,dtk074,win,2016,1552,1,-1,1. d4 { [%clk 0:05:00] } 1... g6 { [%clk 0:05:...
6,7,Rapid,Standard,2021.06.06,06:47:43,300,8,D00,Queen's Pawn Game,Normal,https://lichess.org/tPD6NVEY,False,dtk074,win,2015,1553,1,-1,1. d4 { [%clk 0:05:00] } 1... d5 { [%clk 0:05:...
7,8,Rapid,Standard,2021.06.06,06:42:30,300,8,A40,Modern Defense,Normal,https://lichess.org/eZaTdc66,True,dtk074,win,2014,1553,1,0,1. d4 { [%eval 0.0] [%clk 0:05:00] } 1... g6 {...
8,9,Rapid,Standard,2021.06.06,06:26:58,300,8,B22,"Sicilian Defense: Alapin Variation, Stoltz Attack",Normal,https://lichess.org/74ntZMbF,False,LynneKolar,win,2012,1753,2,-2,1. e4 { [%eval 0.24] [%clk 0:05:00] } 1... c5 ...
9,10,Rapid,Standard,2021.06.05,06:16:37,300,8,C34,King's Gambit Accepted: Fischer Defense,Normal,https://lichess.org/XH3Ag2Xt,False,karthiks,win,2008,1823,4,-3,1. e4 { [%eval 0.24] [%clk 0:05:00] } 1... e5 ...


In [54]:
# take first row and make it into moves

#split game into moves
moves = pd.DataFrame(columns = ['game_id', 'move_number', 'white_move', 'white_eval','white_move_time','black_move','black_eval','black_move_time'])

for x in range(0,games.shape[0]):
    
    game_moves = len(games["moves"][x].split("}"))

# for each item in list add: move number, white move, black move, white eval, black eval
    move_counter = 1
    
    if x%50==0:
        print(x)
    
    if 'eval' in games["moves"][x] or 'clk' in games["moves"][x]:

        for n in range(game_moves):
            
            if 'eval' in games["moves"][x].split("}")[n]:
                try:
                    eval_raw = re.search("\[\%eval (-*[0-9]+\.[0-9]+|\#-*[0-9]+)\]",games["moves"][x].split("}")[n]).group(0)
                except AttributeError:
                    print("x:",x,"n:",n)
                _eval = eval_raw.replace('[%eval ', '').replace(']','')
            else:
                _eval = ''
                
            if 'clk' in games["moves"][x].split("}")[n]:
                clk_raw = re.search("\[\%clk [0-9]+:[0-9]+:[0-9]+\]",games["moves"][x].split("}")[n]).group(0)
                clk = clk_raw.replace('[%clk ', '').replace(']','')
            else:
                clk = ''
            
            temp_move = games["moves"][x].split("}")[n].split(" ")

            if move_counter == 1:
                white_move = temp_move[1]
                white_eval = _eval
                white_move_time = clk
            elif len(temp_move) == 2:
                pass
            elif move_counter % 2 == 1:
                white_move = temp_move[2]
                white_eval = _eval
                white_move_time = clk
                if move_counter == game_moves:
                    black_move = ''
                    black_eval = ''
                    black_move_time = ''
                    moves = moves.append({'game_id': x+1, 'move_number': move_counter/2, 'white_move': white_move, 'white_eval': white_eval, 'white_move_time': white_move_time, 'black_move': black_move, 'black_eval': black_eval, 'black_move_time': black_move_time}, ignore_index=True)
            elif move_counter % 2 == 0:
                black_move = temp_move[2]
                black_eval = _eval
                black_move_time = clk
                moves = moves.append({'game_id': x+1, 'move_number': move_counter/2, 'white_move': white_move, 'white_eval': white_eval, 'white_move_time': white_move_time, 'black_move': black_move, 'black_eval': black_eval, 'black_move_time': black_move_time}, ignore_index=True)

            move_counter = move_counter + 1
    
    elif 'eval' not in games["moves"][x] and 'clk' not in games["moves"][x]:
               
        temp_move = games["moves"][x].split(".")
        
        for y in range(1, len(temp_move)):
            moves = moves.append({'game_id': x+1, 'move_number': y, 'white_move': temp_move[y].split(" ")[1], 'white_eval': '', 'white_move_time': '', 'black_move': temp_move[y].split(" ")[2], 'black_eval': '', 'black_move_time': ''}, ignore_index=True)
            
            

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [74]:
# set results in black_move to null values
moves.loc[(moves.black_move == '1/2-1/2') | (moves.black_move == '1-0') | (moves.black_move == '0-1'), "black_move"] = ''

In [79]:
# move number to int
moves.move_number = moves.move_number.astype(int)

In [104]:
# add moves to games table
# count by game_id
# df.groupby(['col1','col2']).size()
move_count = moves.groupby(['game_id'])['game_id'].count().reset_index(name="move_count")
games = pd.merge(games, move_count, left_on='id', right_on='game_id')


In [108]:
# drop game_id and moves
games.drop(['game_id', 'moves'], axis = 1, inplace=True)
games.head()

,id,game_type,variant,date_utc,time_utc,seconds,increment,opening_code,opening_name,termination_type,url,white,opp_name,outcome,my_rating,opp_rating,my_rating_diff,opp_rating_diff,move_count
0,1,Rapid,Standard,2021.06.12,07:51:47,300,8,D15,Slav Defense: Schlechter Variation,Normal,https://lichess.org/MJtPxMic,True,Ramses_1,win,2031,2108,8,-7,23
1,2,Correspondence,Standard,2021.06.12,07:01:26,-,-,D07,"Queen's Gambit Declined: Chigorin Defense, Exc...",Normal,https://lichess.org/X5pBsOro,True,lichess AI level 6,win,1500,?,NaN,NaN,69
2,3,Rapid,Standard,2021.06.09,07:44:09,300,5,D08,"Queen's Gambit Declined: Albin Countergambit, ...",Normal,https://lichess.org/pYEblF9q,False,rivergrove,win,2027,1907,4,-4,28
3,4,Rapid,Standard,2021.06.09,07:09:47,300,8,C02,"French Defense: Advance Variation, Paulsen Attack",Normal,https://lichess.org/4YNJRTP4,True,Hovhannes1982,win,2019,2076,8,-6,30
4,5,Rapid,Standard,2021.06.08,07:18:53,300,8,B20,Sicilian Defense: Bowdler Attack,Normal,https://lichess.org/5d84RUir,False,rivergrove,win,2017,1711,2,-2,34


In [118]:
# add id to moves table
moves = moves.reset_index()

,index,game_id,move_number,white_move,white_eval,white_move_time,black_move,black_eval,black_move_time
0,0,1,1,d4,0.0,0:05:00,d5,0.27,0:05:00
1,1,1,2,c4,0.0,0:05:06,c6,0.24,0:05:06
2,2,1,3,Nc3,0.0,0:05:12,Nf6,0.13,0:05:13
3,3,1,4,Nf3,0.0,0:05:16,g6,0.69,0:05:19
4,4,1,5,Bf4,0.0,0:05:13,Bg7,0.32,0:05:26
...,...,...,...,...,...,...,...,...,...
33314,33314,969,23,Rf1,,,Qg7,,
33315,33315,969,24,Bg4,,,Re8,,
33316,33316,969,25,Ne3,,,d5,,
33317,33317,969,26,Nxd5,,,c6,,


In [122]:
# change index to id
moves.rename({'index': 'id'}, axis=1, inplace=True)

In [124]:
# add 1 to every value
moves.id += 1

In [20]:
# change date column to date format
games.date_utc = games.date_utc.str.replace(".","-",regex=False)

In [60]:
normal_games = games[games.game_type.isin(['Rapid', 'Correspondence', 'Blitz', 'Bullet','Hyper', 'Classical'])]